<a href="https://colab.research.google.com/github/shivadar/csci226_sql-lab/blob/main/csci226_SQL_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install psycopg2-binary pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 29.1 MB/s eta 0:00:00


In [14]:
import psycopg2, pandas as pd

conn = psycopg2.connect(
    dbname="neondb",
    user="neondb_owner",
    password="npg_rjqA2Ucg7IoY",
    host="ep-empty-sun-ad6wkfza-pooler.c-2.us-east-1.aws.neon.tech.",
    port="5432"
)

cur = conn.cursor()
print("Connected successfully!")

Connected successfully!


In [4]:
cur.execute("""
CREATE TABLE Classes (
    class VARCHAR(50) PRIMARY KEY,
    type VARCHAR(20),
    country VARCHAR(50),
    numGuns INTEGER,
    bore DECIMAL(4,1),
    displacement INTEGER
);
""")

conn.commit()
print("Classes table created successfully!")

Classes table created successfully!


In [5]:
cur.execute("""
CREATE TABLE Ships (
    name VARCHAR(50) PRIMARY KEY,
    class VARCHAR(50),
    launched INTEGER,
    FOREIGN KEY (class) REFERENCES Classes(class)
);
""")

conn.commit()
print("Ships table created successfully!")

Ships table created successfully!


In [8]:
# Insert data into Classes table
cur.execute("""
INSERT INTO Classes VALUES
('Bismarck', 'bb', 'Germany', 8, 15.0, 42000),
('Iowa', 'bb', 'USA', 9, 16.0, 46000),
('Kongo', 'bc', 'Japan', 8, 14.0, 32000),
('North Carolina', 'bb', 'USA', 9, 16.0, 37000),
('Revenge', 'bb', 'Gt. Britain', 8, 15.0, 29000),
('Renown', 'bc', 'Gt. Britain', 6, 15.0, 32000),
('Tennessee', 'bb', 'USA', 12, 14.0, 32000),
('Yamato', 'bb', 'Japan', 9, 18.1, 65000);
""")

# Insert data into Ships table
cur.execute("""
INSERT INTO Ships VALUES
('California', 'Tennessee', 1921),
('Haruna', 'Kongo', 1916),
('Hiei', 'Kongo', 1914),
('Iowa', 'Iowa', 1943),
('Kirishima', 'Kongo', 1915),
('Kongo', 'Kongo', 1913),
('Missouri', 'Iowa', 1944),
('Musashi', 'Yamato', 1942),
('New Jersey', 'Iowa', 1943),
('North Carolina', 'North Carolina', 1941),
('Ramillies', 'Revenge', 1917),
('Renown', 'Renown', 1916),
('Repulse', 'Renown', 1916),
('Resolution', 'Revenge', 1916),
('Revenge', 'Revenge', 1916),
('Royal Oak', 'Revenge', 1916),
('Royal Sovereign', 'Revenge', 1916),
('Tennessee', 'Tennessee', 1920),
('Washington', 'North Carolina', 1941),
('Wisconsin', 'Iowa', 1944),
('Yamato', 'Yamato', 1941);
""")

conn.commit()
print("Data inserted successfully!")


Data inserted successfully!


In [9]:
# Query 1 Find all ships launched before 1921
cur.execute("SELECT name, launched FROM Ships WHERE launched < 1921 ORDER BY launched;")
results1 = cur.fetchall()
df1 = pd.DataFrame(results1, columns=["Ship Name", "Year Launched"])
print("Query 1 - Ships launched before 1921:")
print(df1)


Query 1 - Ships launched before 1921:
          Ship Name  Year Launched
0             Kongo           1913
1              Hiei           1914
2         Kirishima           1915
3            Haruna           1916
4         Royal Oak           1916
5   Royal Sovereign           1916
6            Renown           1916
7           Repulse           1916
8        Resolution           1916
9           Revenge           1916
10        Ramillies           1917
11        Tennessee           1920


In [10]:
# Query 2 Find ship names and their displacement for all battleships
cur.execute("""
SELECT Ships.name, Classes.displacement
FROM Ships, Classes
WHERE Ships.class = Classes.class
AND Classes.type = 'bb'
ORDER BY Classes.displacement DESC;
""")
results2 = cur.fetchall()
df2 = pd.DataFrame(results2, columns=["Ship Name", "Displacement"])
print("Query 2 - Battleships and their displacement:")
print(df2)


Query 2 - Battleships and their displacement:
          Ship Name  Displacement
0           Musashi         65000
1            Yamato         65000
2          Missouri         46000
3        New Jersey         46000
4         Wisconsin         46000
5              Iowa         46000
6    North Carolina         37000
7        Washington         37000
8        California         32000
9         Tennessee         32000
10  Royal Sovereign         29000
11        Ramillies         29000
12          Revenge         29000
13       Resolution         29000
14        Royal Oak         29000


In [15]:
# Query 3 Find the average number of guns by country
cur.execute("""
SELECT Classes.country, ROUND(AVG(Classes.numGuns::numeric), 2) as avg_guns
FROM Classes, Ships
WHERE Ships.class = Classes.class
GROUP BY Classes.country
ORDER BY avg_guns DESC;
""")
results3 = cur.fetchall()
df3 = pd.DataFrame(results3, columns=["Country", "Average Guns"])
print("Query 3 - Average guns by country:")
print(df3)


Query 3 - Average guns by country:
       Country Average Guns
0          USA         9.75
1        Japan         8.33
2  Gt. Britain         7.43


In [12]:
# Query 4 Find all Japanese ships with their class information
cur.execute("""
SELECT Ships.name, Ships.class, Classes.type, Classes.numGuns, Ships.launched
FROM Ships, Classes
WHERE Ships.class = Classes.class
AND Classes.country = 'Japan'
ORDER BY Ships.launched;
""")
results4 = cur.fetchall()
df4 = pd.DataFrame(results4, columns=["Ship Name", "Class", "Type", "Number of Guns", "Year Launched"])
print("Query 4 - Japanese ships with class details:")
print(df4)


Query 4 - Japanese ships with class details:
   Ship Name   Class Type  Number of Guns  Year Launched
0      Kongo   Kongo   bc               8           1913
1       Hiei   Kongo   bc               8           1914
2  Kirishima   Kongo   bc               8           1915
3     Haruna   Kongo   bc               8           1916
4     Yamato  Yamato   bb               9           1941
5    Musashi  Yamato   bb               9           1942
